# Hướng dẫn đặt tên file

#Tên file ipynb đặt theo format: 420300411603_1_10_222222_NguyenVanA
* Lớp: 422001503002
* Ca 1 (1: sáng, 2: chiều)
* STT: 5
* MSSV: 21064051
* Họ tên: NguyenHungAnh


# Timeline
## Chú ý: Phần nộp bài sẽ mở đúng thời gian tương ứng, ngoài thời gian trên, sinh viên sẽ không thể nộp được bài

# Sáng:
* 7:45-8:15: Tích hợp code (30')
* 8:15-8:25: Nộp mô hình (10')
* 8:25-8:45: Nộp báo cao (20')

# Chiều:
* 13:45-14:15: Tích hợp code (30')
* 14:15-14:25: Nộp mô hình (10')
* 14:25-14:45: Nộp báo cao (20')

# Hướng dẫn thực hành
### 1. Ghi đầy đủ thông tin sinh viên vào ô thông tin
### 2. Chạy thực nghiệm mô hình
* Sinh viên có 30 phút để tích hợp hàm eval_caption vào code mô hình của mình để xuất ra độ chính xác của mô hình. Sinh viên thao tác ở Cell "# Load mô hình đã train và tích hợp code (dùng cho Dữ liệu thử)".
* Sinh viên sử dụng dữ liệu thử test.zip tại mục "Dữ liệu thử" để thử xem code chắc chắn chạy được. Yêu cầu:
  - Dữ liệu thử chứa một ảnh và các câu chú thích trong tập tin .txt.
  - Dùng hàm eval_caption để tính và in độ chính xác giữa câu chú thích dự đoán từ mô hình so với từng câu chú thích trong tập tin .txt.
  - In độ chính xác cuối cùng là độ chính xác cao nhất.
* Sau 30 phút, dữ liệu thi sẽ mở tại mục "Dữ liệu thi cuối kỳ". Sinh viên thao tác ở Cell "# In kết quả (dùng cho Dữ liệu thi cuối kỳ)". Sinh viên có 10 phút để hoàn thành các thao tác sau:
  - Thay thế dữ liệu thi vào code.
  - Dùng hàm eval_caption dự đoán mô hình để in ra các độ chính xác (in độ chính xác cuối cùng là độ chính xác cao nhất).
  - Nộp ipynb vừa chạy xong với tên theo quy định theo phần hướng dẫn đặt tên file và nộp vào mục "Nộp mô hình cuối kỳ (Ca thi)".
  
  # Chú ý: mục "Nộp mô hình cuối kỳ" chỉ được mở trong vòng 10 phút.  

### 3. Trả lời phần "Câu hỏi" theo mẫu
* Sinh viên có 20 phút để hoàn thành, đặt tên theo quy định và nộp vào mục "Nộp báo cáo cuối kỳ (Ca thi)".
  # Chú ý: mục "Nộp báo cáo cuối kỳ" chỉ được mở trong vòng 20 phút.  


# Mô hình

In [ ]:
# Code cho mô hình
# Tất cả code mô hình của sinh chuẩn bị (đã up lên LMS từ trước) được ở đây
# Sinh viên có thể tùy ý tạo thêm cell mới ở phần Mô hình

In [14]:
import pandas as pd
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import keras
import re
import nltk
from nltk.corpus import stopwords
import string
import json
from time import time
import pickle
from keras.preprocessing import image
from keras.models import Model, load_model
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from keras.layers import Input, Dense, Dropout, Embedding, LSTM
from keras.layers import add
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input

from google.colab import drive
drive.mount('/content/drive')

# Tải mô hình đã huấn luyện
model_lstm = load_model('/content/drive/MyDrive/Colab_Notebooks/deep_learning/ImageCaptionv2/image_captioning_model.keras')

# Load từ điển từ/idx và idx/to từ
with open('/content/drive/MyDrive/Colab_Notebooks/deep_learning/ImageCaptionv2/word_to_idx.json', 'r') as f:
    word_to_idx = json.load(f)

with open('/content/drive/MyDrive/Colab_Notebooks/deep_learning/ImageCaptionv2/idx_to_word.json', 'r') as f:
    idx_to_word = json.load(f)

# Chuẩn bị mô hình CNN với InceptionV3
model_cnn = InceptionV3(weights='imagenet')
model_cnn = Model(model_cnn.input, model_cnn.layers[-2].output)

# Đặt các thông số cần thiết
max_len = 49

# Hàm tiền xử lý ảnh
def preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(299,299))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return x

def encode_image(img_path):
    img = preprocess_image(img_path)
    feature_vector = model_cnn.predict(img, verbose=0)
    feature_vector = feature_vector.reshape(feature_vector.shape[1],)
    return feature_vector

def predict_caption(path):
    encoding_test_indiv = {}
    encoding_test_indiv[0] = encode_image(path)

    #reshape phù hợp với mô hình LSTM
    photo = encoding_test_indiv[0].reshape((1,2048))
    # i = plt.imread(path)
    # # Hiển thị ảnh
    # plt.imshow(i)
    # plt.axis("off")
    # plt.show()
    in_text = 'startseq'
    for _ in range(max_len):
        sequence = [word_to_idx[w] for w in in_text.split() if w in word_to_idx]
        sequence = pad_sequences([sequence], maxlen=max_len, padding='post')
        yhat = model_lstm.predict([photo, sequence], verbose=0) #tắt hiện dòng predict processing bar
        yhat = np.argmax(yhat)
        word = idx_to_word.get(str(yhat)) # vì dữ liệu là dạng chữ số nên phải chuyển về dạng số mới get
        if word is None:
            break
        in_text += ' ' + word
        if word == 'endseq':
            break
    final = in_text.split()
    final = final[1:-1]
    final = ' '.join(final)
    return final

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
img = '/content/drive/MyDrive/Colab_Notebooks/deep_learning/000000000724.jpg'
caption = predict_caption(img)
print(caption)

stop sign with stop sign on it


# Tích hợp code

In [17]:
# Hàm đánh giá

from sentence_transformers import SentenceTransformer, util

s1 = "Tôi thích ăn phở."
s2 = "Tôi yêu món phở."

def eval_caption(sentence1, sentence2):
    model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
    # Encode sentences
    embeddings = model.encode([sentence1, sentence2], convert_to_tensor=True)
    # Compute cosine similarity
    similarity = util.pytorch_cos_sim(embeddings[0], embeddings[1])
    print(f"Similarity: {similarity.item():.4f}")

eval_caption(s1, s2)

Similarity: 0.5653


In [16]:
# Load mô hình đã train và tích hợp code (dùng cho Dữ liệu thử)
# Sinh viên không được tạo thêm cells mới
img = '/content/drive/MyDrive/Colab_Notebooks/deep_learning/000000000724.jpg'
caption = predict_caption(img)

s1 = caption
s2 = "A stop sign that is hanging upside down." #Cai cao nhat
eval_caption(s1, s2)

Similarity: 0.7473


# Kết quả

In [23]:
# In kết quả (dùng cho Dữ liệu thi cuối kỳ)
# Sinh viên không được tạo thêm cells mới
img = '/content/drive/MyDrive/Colab_Notebooks/deep_learning/000000000885.jpg'
caption = predict_caption(img)

s1 = caption
s2 = "A man holding a tennis racket playing tennis." #Cai cao nhat
eval_caption(s1, s2)


Similarity: 0.8969


# Câu hỏi

Khối 1: Trích xuất đặc trưng hình ảnh bằng CNN

* Nhiệm vụ: Chuyển ảnh đầu vào thành vector đặc trưng có nghĩa

* Vị trí code: Cell 7, dòng 10–35

* Input: Ma trận ảnh RGB (shape: H x W x 3)

* Output: Vector đặc trưng shape: 1 x 2048

Khối 2: Tiền xử lý caption và ánh xạ từ/câu sang vector

* Nhiệm vụ: Mã hóa caption bằng tokenizer và chuyển thành chuỗi số, sau đó nhúng thành embedding vectors

* Vị trí code: Cell 9, dòng 5–40

* Input: Chuỗi văn bản (caption)

* Output: Dãy số (chuỗi token), rồi embedding vector bằng glove.5b.txt

Khối 3: Decoder LSTM

* Nhiệm vụ: Sinh caption từ đặc trưng ảnh và thông tin ngữ cảnh

* Vị trí code: Cell 11, dòng 15–60

* Input: Vector đặc trưng từ ảnh + caption đã mã hóa

* Output: Xác suất phân phối trên từ tiếp theo trong câu (softmax over vocabulary)

Khối 4: Hậu xử lý và tạo câu hoàn chỉnh

* Nhiệm vụ: Duyệt qua kết quả đầu ra để tạo câu mô tả ảnh

* Vị trí code: Cell 13, dòng 5–20

* Input: Dãy xác suất (từng bước)

* Output: Caption đầy đủ (chuỗi từ)